In [155]:
# Import libraries

import sys
import logging
import pandas as pd
import numpy as np
import xarray as xr
from pathlib import Path

root_path = Path(globals()['_dh'][0]).resolve().parent
sys.path.append(str(root_path))

import paths
from library.assumptions import read_assumptions
from library.weather import load_weather
from library.network import build_network

logging.basicConfig(level=logging.INFO)

In [28]:
# Set the configuration

## Parameters you won't change very often
base_currency = 'SEK'
exchange_rates = {
    'EUR': 11.68,
    'USD': 10.70
}
base_year = 2024
discount_rate = 0.05
onwind_turbine =  "2030_5MW_onshore.yaml"
offwind_turbine = "2030_20MW_offshore.yaml"
resolution = 3

## Parameters that will change frequently
target_year = 2030
use_offwind = True
use_h2 = True
h2_initial = 1000
biogas_limit = 500
load_target = 19 #TWh

In [29]:
# Load the data needed from assumptions, the electricity demand, and the atlite output from ERA5 weather data for VGR 2023

## Transform assumptions to range base_year to target_year
assumptions = read_assumptions(paths.input_path / 'assumptions.csv', base_year, target_year, base_currency, exchange_rates, discount_rate)

# Read the normalized demand from csv file (see normalize_demand() in library.demand for details)
# And then calculate target_load using projection of energy need in target_year
normalized_demand = pd.read_csv(paths.input_path / 'demand/normalized-demand-2023-3h.csv', delimiter=',')
target_load = load_target * normalized_demand['value'].values.flatten() * 1_000_000

# Create of load the cutout from atlite (we assume weather data from 2023 and a 3h window)
geo = '14' # All of VGR
section = None
cutout, selection, index = load_weather(geo, section, '2023-01', '2023-12')
geography = selection.total_bounds  

capacity_factor_solar = xr.open_dataarray(paths.input_path / 'renewables' / f"capacity-factor-{geo}-2023-01-2023-12-solar.nc").values.flatten()
capacity_factor_onwind = xr.open_dataarray(paths.input_path / 'renewables' / f"capacity-factor-{geo}-2023-01-2023-12-onwind.nc").values.flatten()
capacity_factor_offwind = xr.open_dataarray(paths.input_path / 'renewables' / f"capacity-factor-{geo}-2023-01-2023-12-offwind.nc").values.flatten()

In [30]:
# Build the network

network = build_network(index, resolution, geography, target_load, assumptions, capacity_factor_solar, capacity_factor_onwind, capacity_factor_offwind, use_offwind, use_h2, h2_initial, biogas_limit)

In [31]:
# Add constraints to the model and run the optimization

## Create the model
model = network.optimize.create_model()

generator_capacity = model.variables["Generator-p_nom"]
link_capacity = model.variables["Link-p_nom"]

## Add offwind constraint
if use_offwind:
    offwind_percentage = 0.5

    offwind_constraint = (1 - offwind_percentage) / offwind_percentage * generator_capacity.loc['offwind'] - generator_capacity.loc['onwind']
    model.add_constraints(offwind_constraint == 0, name="Offwind_constraint")

## Add battery charge/discharge ratio constraint
lhs = link_capacity.loc["battery-discharge"] - network.links.at["battery-charge", "efficiency"] * link_capacity.loc["battery-discharge"]
model.add_constraints(lhs == 0, name="Link-battery_fix_ratio")

## Run optimization
network.optimize.solve_model(solver_name='highs')

INFO:linopy.model: Solve problem using Highs solver
INFO:linopy.io:Writing objective.
Writing integer variables.: 100%|██████████| 2/2 [00:00<00:00, 669.75it/s]
INFO:linopy.io: Writing time: 0.53s
INFO:linopy.solvers:Log file at /tmp/highs.log


Running HiGHS 1.7.2 (git hash: 184e327): Copyright (c) 2024 HiGHS under MIT licence terms


INFO:linopy.constants: Optimization successful: 
Status: ok
Termination condition: optimal
Solution: 46739 primals, 105142 duals
Objective: 4.74e+10
Solver model: available
Solver message: optimal

INFO:pypsa.optimization.optimize:The shadow-prices of the constraints Generator-ext-p-lower, Generator-ext-p-upper, Link-ext-p-lower, Link-ext-p-upper, Store-ext-e-lower, Store-ext-e-upper, Store-energy_balance, Offwind_constraint were not assigned to the network.


Coefficient ranges:
  Matrix [4e-06, 3e+02]
  Cost   [8e+01, 2e+06]
  Bound  [0e+00, 0e+00]
  RHS    [5e+02, 3e+03]
Presolving model
45265 rows, 39432 cols, 115354 nonzeros  0s
34550 rows, 28717 cols, 110506 nonzeros  0s
33920 rows, 28087 cols, 111114 nonzeros  0s

Solving MIP model with:
   33920 rows
   28087 cols (0 binary, 5 integer, 0 implied int., 28082 continuous)
   111114 nonzeros

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
     Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0       0         0   0.00%   -inf            inf                  inf        0      0      0         0     0.4s
         0       0         0   0.00%   36119724915.4   inf                  inf        0      0      1     26196    10.7s
 S       0       0         0   0.00%   36120875584.7   47535397607.21    24.01%       55     19     18     26518    1

('ok', 'optimal')

In [32]:
network.statistics()

Optimal Capacity  Installed Capacity        Supply  \
Generator backstop       2384.508785                 0.0  4.812068e+05   
          biogas          500.000000                 0.0  4.662297e+05   
          offwind        9307.902857                 0.0  1.677123e+06   
          onwind         9307.902857                 0.0  1.311354e+07   
          solar          4576.454820                 0.0  4.843298e+06   
Link      AC             3495.106319                 0.0  1.758867e+07   
          biogas          500.000000                 0.0  4.662297e+05   
          h2             2400.000000                 0.0  2.308374e+06   
          mixedgas       1800.000000                 0.0  9.301193e+05   
Load      -                 0.000000                 0.0  0.000000e+00   
Store     h2            24722.177408                 0.0  1.154187e+06   
          li-ion            0.000000                 0.0  1.308221e-12   

                      Withdrawal      Dispatch  Transmission  Capacity Factor  \
Generator backstop  0.000000e+00  4.812068e+05  0.000000e+00         0.023037   
          biogas    0.000000e+00  4.662297e+05  0.000000e+00         0.106445   
          offwind   0.000000e+00  1.677123e+06  0.000000e+00         0.020569   
          onwind    0.000000e+00  1.311354e+07  0.000000e+00         0.160829   
          solar     0.000000e+00  4.843298e+06  0.000000e+00         0.120811   
Link      AC        1.758867e+07  0.000000e+00  1.758867e+07         0.574472   
          biogas    4.662297e+05  0.000000e+00  4.662297e+05         0.106445   
          h2        3.199478e+06 -8.911042e+05  0.000000e+00         0.152182   
          mixedgas  1.620417e+06 -6.902976e+05  1.620417e+06         0.102766   
Load      -         1.900000e+07 -1.900000e+07  0.000000e+00              NaN   
Store     h2        1.154187e+06  7.639755e-10  0.000000e+00         0.417991   
          li-ion    1.308535e-12 -3.140232e-16  0.000000e+00              NaN   

                     Curtailment  Capital Expenditure  \
Generator backstop  0.000000e+00         0.000000e+00   
          biogas    0.000000e+00         0.000000e+00   
          offwind   3.395359e+07         2.135240e+10   
          onwind    1.809409e+07         1.254669e+10   
          solar     3.265928e+05         2.377129e+09   
Link      AC        0.000000e+00         0.000000e+00   
          biogas    0.000000e+00         0.000000e+00   
          h2        0.000000e+00         6.546700e+08   
          mixedgas  0.000000e+00         2.396956e+09   
Load      -         0.000000e+00         0.000000e+00   
Store     h2        0.000000e+00         1.884745e+09   
          li-ion    0.000000e+00         0.000000e+00   

                    Operational Expenditure  Revenue  Market Value  
Generator backstop             5.058446e+09      0.0           NaN  
          biogas               5.957205e+08      0.0           NaN  
          offwind              9.699640e+07      0.0           NaN  
          onwind               3.774022e+08      0.0           NaN  
          solar                0.000000e+00      0.0           NaN  
Link      AC                   0.000000e+00      0.0           NaN  
          biogas               0.000000e+00      0.0           NaN  
          h2                   0.000000e+00      0.0           NaN  
          mixedgas             1.042828e+08      0.0           NaN  
Load      -                    0.000000e+00      0.0           0.0  
Store     h2                   0.000000e+00      0.0           NaN  
          li-ion              -8.809448e-15      0.0           NaN

In [34]:
network.statistics.capex()

component  carrier 
Link       h2          6.546700e+08
           mixedgas    2.396956e+09
Store      h2          1.884745e+09
Generator  offwind     2.135240e+10
           onwind      1.254669e+10
           solar       2.377129e+09
dtype: float64

In [ ]:
solar_curtailment = NETWORK.generators_t.p_max_pu[['Solar park']] * NETWORK.generators.loc['Solar park']['p_nom_opt'] - NETWORK.generators_t.p[['Solar park']]
solar_curtailment['Curtailment'] = solar_curtailment['Solar park']
solar_curtailment.drop(columns=['Solar park'], inplace=True)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 6))

pd.concat([NETWORK.links_t.p0[['Solar battery link', 'Solar H2 link', 'Solar load link']], solar_curtailment]).resample('ME').sum().plot(ax=ax1, kind='bar', stacked=True, legend=True, alpha=0.2)

wind_curtailment = NETWORK.generators_t.p_max_pu[['Onwind park']] * NETWORK.generators.loc['Onwind park']['p_nom_opt'] - NETWORK.generators_t.p[['Onwind park']]
wind_curtailment['Curtailment'] = wind_curtailment['Onwind park']
wind_curtailment.drop(columns=['Onwind park'], inplace=True)

pd.concat([NETWORK.links_t.p0[['Onwind battery link', 'Onwind H2 link', 'Onwind load link']], wind_curtailment]).resample('ME').sum().plot(ax=ax2, kind='bar', stacked=True, legend=True, alpha=0.2)

In [ ]:
data = pd.concat([NETWORK.links_t.p0[['Solar battery link', 'Solar H2 link', 'Solar load link']], solar_curtailment]).resample('ME').sum()

# Prepare data for Plotly
data = data.reset_index()

# Create an interactive bar chart using Plotly
fig = go.Figure()

for column in data.columns[1:]:  # Skip the first column as it is the index
    fig.add_trace(go.Bar(
        x=data['snapshot'],  # Use the reset index for the x-axis
        y=data[column],
        name=column
    ))

# Update layout for better visualization
fig.update_layout(
    barmode='stack',
    xaxis_title='Date',
    yaxis_title='Energy (MWh)',
    title='Monthly Energy Flows and Curtailment',
    legend_title='Components',
    xaxis=dict(tickformat="%Y-%m"),
    hovermode='x unified'
)

# Show the interactive chart
fig.show()

In [ ]:
data = pd.concat([NETWORK.links_t.p0[['Solar battery link', 'Solar H2 link', 'Solar load link']], solar_curtailment]).resample('ME').sum()

# Prepare data for Plotly
data = data.reset_index()

# Create an interactive bar chart using Plotly
fig = go.Figure()

for column in data.columns[1:]:  # Skip the first column as it is the index
    fig.add_trace(go.Bar(
        x=data['snapshot'],  # Use the reset index for the x-axis
        y=data[column],
        name=column
    ))

# Update layout for better visualization
fig.update_layout(
    barmode='stack',
    xaxis_title='Date',
    yaxis_title='Energy (MWh)',
    title='Monthly Energy Flows and Curtailment',
    legend_title='Components',
    xaxis=dict(tickformat="%Y-%m"),
    hovermode='x unified'
)

# Show the interactive chart
fig.show()

curtailment = NETWORK.generators_t.p_max_pu[['Onwind park']] * NETWORK.generators.loc['Onwind park']['p_nom_opt'] - NETWORK.generators_t.p[['Onwind park']]
curtailment['Curtailment'] = curtailment['Onwind park']
curtailment.drop(columns=['Onwind park'], inplace=True)

fig1, ax1 = plt.subplots(figsize=(24, 6))
pd.concat([NETWORK.links_t.p0[['Onwind battery link', 'Onwind H2 link', 'Onwind load link']], curtailment]).resample('ME').sum().plot(ax=ax1, kind='bar', stacked=True, legend=True, alpha=0.2)